<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Lab: Cleaning Rock Song Data

_Authors: Dave Yerrington (SF)_

---


In [211]:
import sqlite3

In [212]:
import pandas as pd
import numpy as np 
import seaborn as sns

%matplotlib inline

### 1. Load `rock.csv` and do an initial examination of its data columns.

In [213]:
rockfile = "datasets/rock.csv"

In [214]:
# Load the data.
rockfile_df = pd.read_csv(rockfile)
rockfile_df.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


In [246]:
# Look at the information regarding its columns.
%time
rockfile_df.info()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 26 µs
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2229 entries, 0 to 2229
Data columns (total 8 columns):
Song           2229 non-null object
Artist         2229 non-null object
Release        1651 non-null float64
Song_Artist    2229 non-null object
First          2229 non-null int64
Year           2229 non-null int64
Playcount      2229 non-null int64
F_G            2229 non-null int64
dtypes: float64(1), int64(4), object(3)
memory usage: 236.7+ KB


### 2.  Clean up the column names.

Let's clean up the column names. There are two ways we can accomplish this:

#### 2.A Change the column names when you import the data using `pd.read_csv()`.

Notice that, when passing `names=[..A LIST OF STRING..]` with a number of columns that matches the number of strings in the passed list, you replace the column names.

NOTE: When you create custom column names, the first row of the `.csv` already represents a header. It is important to tell `pandas` to skip that row. The `skiprows=1` keyword argument to `read_csv()` will tell `pandas` to skip the first row.

In [216]:
# Change the column names when loading the '.csv':
rockfile_df_1 = pd.read_csv(rockfile, skiprows=1, names=['Song', 'Artist', 'Release_Year', 'Song_Artist', 'First', 'Year', 'Playcount', 'F_G'])
rockfile_df_1.head()

,Song,Artist,Release_Year,Song_Artist,First,Year,Playcount,F_G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.B Change column names using the `.rename()` function.

The `.rename()` function takes an argument, `columns=name_dict`, in which `name_dict` is a dictionary containing the original column names as keys and the new column names as values.

In [217]:
# Change the column names using the `.rename()` function.
new_col = ['Song', 'Artist', 'Release_Year', 'Song_Artist', 'First', 'Year', 'Playcount', 'F_G']
old_col = list(rockfile_3df.columns)
rockfile_df_2 = rockfile_df.rename(columns={o:n for o, n in zip(old_col, new_col)})
rockfile_df_2.head()

,Song,Artist,Release_Year,Song_Artist,First,Year,Playcount,F_G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.C Reassigning the `.columns` attribute of a DataFrame.

You can also just reassign the `.columns` attribute to a list of strings containing the new column names. 

The only caveat with reassigning `.columns` is that you have to reassign all of the column names at once. You can't partially replace a value by working on `.columns` directly. You have to reassign `.columns` with a list of equal length. 

In [218]:
# Replace the column names by reassigning the `.columns` attribute.
rockfile_df.columns = ['Song', 'Artist', 'Release', 'Song_Artist', 'First', 'Year', 'Playcount', 'F_G']
rockfile_df.head()

,Song,Artist,Release,Song_Artist,First,Year,Playcount,F_G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


### 3. Subsetting data where null values exist.

We have mixed `str` and `NaN` values in the `release` column. `NaN` stands for "not a number" and is the way `pandas` handles "nulls" or nonexistent data. We can use the `.isnull()` method of a Series to find null values.

Print the header of the data subset to where the `release` column is null values.

In [247]:
# Show records where df['release'] is null
rockfile_null = rockfile_df[rockfile_df['Release'].isnull()]
rockfile_null.head()


,Song,Artist,Release,Song_Artist,First,Year,Playcount,F_G
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,NaN,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,NaN,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,NaN,Let's Get It Up by AC/DC,1,0,4,0


### 4. Update slices of your DataFrame based on mask selection/slices.

In many scenarios, we want to upate values in our DataFrame according to criteria. Let's say we wanted to set all of the null values in `release` to 0.

With newer versions of `pandas`, in order to manipulate data in the original DataFrame, we have to use `.loc` while performing reassignment using a mask and an index.

For example, the following won't always work:
```python
df[row_mask]['column_name'] = new_value
```

The best way to accomplish the same task is:
```python
df.loc[row_mask, 'column_name'] = new_value
```

For multiple column assignment, you would use:
```python
df.loc[row_mask, ['col_1', 'col_2', 'col_3']] = new_value
```

#### 4.A Let's try it out. Make all of the null values in `release` 0.

In [249]:
# Replace release nulls with 0
# advanced indexing in loc, setting conditions in the row placeholder
rockfile_df.loc[rockfile_df.Release.isnull(), 'Release'] = 0



#### 4.B Verify that `release` contains no null values.

In [251]:
# A:
# summing up the counts of all null values
rockfile_df.isnull().sum()

Song           0
Artist         0
Release        0
Song_Artist    0
First          0
Year           0
Playcount      0
F_G            0
dtype: int64

### 5. Ensure that the data types of the columns make sense. 

Verifying column data types is a critical part of data munging. If columns have the wrong data type, then there is usually corrupted or incorrect data in some of the observations.

#### 5.A Look at the data types for the columns. Are any incorrect given what the data represents?

> * Song, Artist & Song_Artist should be String

> * Release should be Integer

In [222]:
# A:
rockfile_df.dtypes

Song           object
Artist         object
Release        object
Song_Artist    object
First           int64
Year            int64
Playcount       int64
F_G             int64
dtype: object

### 6. Investigate and clean up the `release` column.

The `release` column is a string data type when it should be an integer.

#### 6.A Figure out what value(s) are causing the `release` column to be encoded as a string instead of an integer.

In [252]:
# A:
# Gather unique values
rockfile_df.Release.value_counts().sort_index()
rockfile_df.Release.unique()

array([ 1982.,     0.,  1981.,  1980.,  1975.,  2000.,  2002.,  1992.,
        1985.,  1993.,  1976.,  1995.,  1979.,  1984.,  1977.,  1990.,
        1986.,  1974.,  2014.,  1987.,  1973.,  2001.,  1989.,  1997.,
        1971.,  1972.,  1994.,  1970.,  1966.,  1965.,  1983.,  1955.,
        1978.,  1969.,  1999.,  1968.,  1988.,  1962.,  2007.,  1967.,
        1958.,  1996.,  1991.,  2005.,  2011.,  2004.,  2012.,  2003.,
        1998.,  2008.,  1964.,  2013.,  2006.,  1963.,  1961.])

In [224]:
# SONGFACTS.COM

#### 6.B Look at the rows in which there is incorrect data in the `release` column.

In [225]:
# A:
rockfile_df[pd.to_numeric(rockfile_df['Release'], errors='coerce').isnull()]


,Song,Artist,Release,Song_Artist,First,Year,Playcount,F_G
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


#### 6.C. Clean up the data. Normally we may replace the offending data with null np.nan values, however we previously converted all of the nan values in the release column to zeros so we might as well continue with the same practice. Replacing with 0 (or nan) will allow us to convert the column to numeric.

In [226]:
# A:
anamoly = pd.to_numeric(rockfile_df['Release'], errors='coerce').isnull()
rockfile_df.loc[anamoly, 'Release'] = 0


### 7. Get summary statistics for the `release` column using the `.describe()` function.

Now that the `release` column is finally a numeric data type, we can apply the `.describe()` function.  

#### 7.A Print out the summary stats for the `release` column. What is the earliest and latest release date?

In [227]:
# A:
rockfile_df['Release'] = rockfile_df['Release'].map(lambda x: int(x))
rockfile_df['Release'].describe()

count    2230.000000
mean     1465.331390
std       867.196161
min         0.000000
25%         0.000000
50%      1973.000000
75%      1981.000000
max      2014.000000
Name: Release, dtype: float64

#### 7.B Based on the summary statistics, is there anything else wrong with the `release` column? 

In [228]:
# A:
# the mean looks extremely low which doesnt make sense, there must be an extreme value here
# looks like the mean calculation is including the 0 values... its best if we ignore them
rockfile_df = rockfile_df.drop(rockfile_df[(rockfile_df['Release'] < 1900) & (rockfile_df['Release'] > 0)].index)
rockfile_df['Release'] = rockfile_df['Release'].replace(0, np.nan)
rockfile_df['Release'].describe()

count    1651.000000
mean     1978.569352
std         9.309780
min      1955.000000
25%      1971.000000
50%      1977.000000
75%      1984.000000
max      2014.000000
Name: Release, dtype: float64

_Looking at the DataFrame that contains the year 1071, we can see that the year was probably corrupted and should be replaced with something else if possible._

### 8. Make changes and investigate using custom functions with `.apply()`.

Let's say we want to traverse every single row in our data set and apply a function to that row.

#### 8.A Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970.


In [229]:
# A:
def rockfile_filter(row):
    print('Song: ' + str(row['Song']))
    print('Artist: ' + str(row['Artist']))
    print('Year released: ' + str(row['Release']))
    print('Release date < 1970: ' + str(row['Release'] < 1970))
    print('\n')

#### 8.B Using the `.apply()` function, apply the function you wrote to the first four rows of the DataFrame.

You will need to tell the `apply` function to operate row by row. Setting the keyword argument as `axis=1` indicates that the function should be applied to each row individually.

In [230]:
# A:
rockfile_df.head(4).apply(rockfile_filter, axis=1)

Song: Caught Up in You
Artist: .38 Special
Year released: 1982.0
Release date < 1970: False


Song: Fantasy Girl
Artist: .38 Special
Year released: nan
Release date < 1970: False


Song: Hold On Loosely
Artist: .38 Special
Year released: 1981.0
Release date < 1970: False


Song: Rockin' Into the Night
Artist: .38 Special
Year released: 1980.0
Release date < 1970: False




0    None
1    None
2    None
3    None
dtype: object

You'll notice that there will be a final output Series of `None` values. The `.apply()` function, if a return value is not specified, will return a Series of `None` values (similar to how the default return for Python functions is `None` when a return statement is not specified).

### 9. Write a function that converts cells in a DataFrame to float and otherwise replaces them with `np.nan`.

If applied to our data, it would keep only the numeric information and otherwise input null values.

Recall that the try-except syntax in Python is a great way to try something and take another action if the initial step fails:

```python
try:
    Perform some action.
except:
   Perform some other action if the first failed with an error.
```

#### 9.A Write the function that takes a column and converts all of its values to float if possible and `np.nan` otherwise. The return value should be the converted Series.

In [231]:
# A:
def float_converter(col):
    converted_col = pd.to_numeric(col, errors='coerce', downcast='float')
    print converted_col

# 
def converter_helper(value):
        try

#### 9.B Try your function out on the rock song data and ensure the output is what you expected.


In [244]:
# A:
song_series = rockfile_df['Song'].copy(deep=True)
float_converter(song_series)


0                                  Caught Up in You
1                                      Fantasy Girl
2                                   Hold On Loosely
3                            Rockin' Into the Night
4                                 Art For Arts Sake
5                                        Kryptonite
6                                             Loser
7                                     When I'm Gone
8                                        What's Up?
9                                        Take On Me
10                            Baby, Please Don't Go
11                                    Back In Black
12                                          Big Gun
13                           CAN'T STOP ROCK'N'ROLL
14                      Dirty Deeds Done Dirt Cheap
15                          For Those About To Rock
16                                 Girls Got Rhythm
17                                   Hard As A Rock
18                               Have a Drink On Me
19          

#### 9.C Describe the new float-only DataFrame.

In [233]:
# A: